In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-the-success-of-bank-telemarketing/sample_submission.csv
/kaggle/input/predict-the-success-of-bank-telemarketing/train.csv
/kaggle/input/predict-the-success-of-bank-telemarketing/test.csv


In [2]:
# Load the Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score
from datetime import datetime
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
# Load the data
train_data = pd.read_csv('/kaggle/input/predict-the-success-of-bank-telemarketing/train.csv')
test_data = pd.read_csv('/kaggle/input/predict-the-success-of-bank-telemarketing/test.csv')


 # **EDA**

In [4]:
# print(train_data.head())
# print(test_data.head())

In [5]:
# # Basic info 
# print(train_data.info())

In [6]:
# # Summary statistics
# print(train_data.describe())


# #Check for Missing Values
# print(train_data.isnull().sum())

# # Can clearly Comment that poutcome and contact has the highest number of NULL values

In [7]:
# print(train_data['target'].value_counts())

In [8]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Set general figure size for all plots
# plt.figure(figsize=(15, 12))
# train= train_data

# # 1. Distribution of Target Variable (Subscribed or Not)
# plt.subplot(3, 3, 1)
# sns.countplot(x='target', data=train, palette='Set2')
# plt.title('Distribution of Target Variable')
# plt.xlabel('Subscribed to Term Deposit')
# plt.ylabel('Count')

# # 2. Age Distribution
# plt.subplot(3, 3, 2)
# sns.histplot(train['age'], bins=20, kde=True, color='purple')
# plt.title('Age Distribution')
# plt.xlabel('Age')
# plt.ylabel('Frequency')

# # 3. Balance Distribution
# plt.subplot(3, 3, 3)
# sns.histplot(train['balance'], bins=30, kde=True, color='green')
# plt.title('Balance Distribution')
# plt.xlabel('Balance (Euros)')
# plt.ylabel('Frequency')

# # 4. Marital Status vs Subscription
# plt.subplot(3, 3, 4)
# sns.countplot(x='marital', hue='target', data=train, palette='Set1')
# plt.title('Marital Status vs Subscription')
# plt.xlabel('Marital Status')
# plt.ylabel('Count')

# # 5. Education vs Subscription
# plt.subplot(3, 3, 5)
# sns.countplot(x='education', hue='target', data=train, palette='Set3')
# plt.title('Education Level vs Subscription')
# plt.xlabel('Education Level')
# plt.ylabel('Count')
# plt.xticks(rotation=45)

# # 6. Housing Loan vs Subscription
# plt.subplot(3, 3, 6)
# sns.boxplot(data=train, x='target', y='balance', palette='Set1')
# plt.yscale('log') 
# plt.title("Balance Distribution by Subscription Status")
# plt.xlabel("Subscribed to Term Deposit")
# plt.ylabel("Balance (Euros)")


# # 7. Previous Outcome vs Subscription
# plt.subplot(3, 3, 7)
# sns.countplot(x='poutcome', hue='target', data=train, palette='viridis')
# plt.title('Previous Campaign Outcome vs Subscription')
# plt.xlabel('Previous Campaign Outcome')
# plt.ylabel('Count')

# # 8. Age vs Balance (Scatter Plot)
# plt.subplot(3, 3, 8)
# sns.countplot(x='housing', hue='target', data=train, palette='coolwarm')
# plt.title('Housing Loan vs Subscription')
# plt.xlabel('Has Housing Loan')
# plt.ylabel('Count')


# # 9. Last Contact Duration vs Subscription
# plt.subplot(3, 3, 9)
# sns.boxplot(x='target', y='duration', data=train, palette='Set1')
# plt.title('Last Contact Duration vs Subscription')
# plt.xlabel('Subscribed to Term Deposit')
# plt.ylabel('Duration of Last Contact (Seconds)')
# train_data = train 

# # Adjust layout to prevent overlap
# plt.tight_layout()

# # Show all plots
# plt.show()

In [9]:
###INSIGHTS

# People who have more balance in their account they are showing more interest in subscirbing the term deposit
# Those who are single are subscribing more comparative to married people
# People have have tertiary level oof education, they are subscribing FD more
# If previos outcome is a success then no. of people subscribed it more , intersting is failure
# of previous outcome is also resulting in YES manytimes

# Those whose calls cut in less than 300-350 seconds, mostly they have not subscribed although there are many exceptions


In [10]:
# plt.figure(figsize=(6, 4))
# sns.countplot(data=train_data, x='job', hue='target', palette='Set2')
# plt.title("Job Type vs. Subscription Rate")
# plt.xlabel("Job Type")
# plt.ylabel("Count")
# plt.xticks(rotation=45)
# plt.legend(title='Subscribed to Term Deposit')
# plt.tight_layout()
# plt.show()

# # age_bins = [0, 20, 30, 40, 50, 60, 70, 80, 100]
# # age_labels = ['<20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80+']
# # train_data['age_group'] = pd.cut(train_data['age'], bins=age_bins, labels=age_labels)

# # plt.figure(figsize=(5, 3))
# # sns.countplot(data=train_data, x='age_group', hue='target', palette='cool')
# # plt.title("Age Group vs. Subscription Rate")
# # plt.xlabel("Age Group")
# # plt.ylabel("Count")
# # plt.legend(title='Subscribed to Term Deposit')
# # plt.show()

In [11]:
# ##INSIGHTS 

# People are more from age group 20-60 but the ratio of subscription of Term Deposit is more in <20 or 60+ age .
# blue-collar/labour has least yes ratio whereas the students,self employed ,retired and housemaids has the highest ratio of no vs yes

In [12]:
# # Create a new feature based on age and balance interaction
# train_data['balance_duration'] = train_data['balance'] * train_data['duration']

# # Create a new feature based on age and balance interaction
# test_data['balance_duration'] = test_data['balance'] * test_data['duration']



Tried to make new column/ feature extraction but did not got the good aaccuracy 

In [13]:
# print(train_data.head())

# **Preprocessing**

In [14]:

# Fill missing values for categorical features with 'unknown' and convert to string type
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
train_data[categorical_columns] = train_data[categorical_columns].fillna('unknown').astype(str)
test_data[categorical_columns] = test_data[categorical_columns].fillna('unknown').astype(str)

# Fill missing numerical values with column mean
numerical_columns = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
# train_data[numerical_columns] = train_data[numerical_columns].apply(lambda x: x.fillna(x.mode()[0]))
# test_data[numerical_columns] = test_data[numerical_columns].apply(lambda x: x.fillna(x.mode()[0]))
train_data[numerical_columns] = train_data[numerical_columns].fillna(train_data[numerical_columns].median())
test_data[numerical_columns] = test_data[numerical_columns].fillna(test_data[numerical_columns].median())


# # Add interaction feature between 'balance' and 'duration'
# train_data['balance_duration_interaction'] = train_data['balance'] * train_data['duration']
# test_data['balance_duration_interaction'] = test_data['balance'] * test_data['duration']



if 'last contact date' in train_data.columns:
    # Convert to datetime
    train_data['last contact date'] = pd.to_datetime(train_data['last contact date'], errors='coerce')
    test_data['last contact date'] = pd.to_datetime(test_data['last contact date'], errors='coerce')
    
    # Extract day, month, and year
    train_data['last_contact_day'] = train_data['last contact date'].dt.day
    train_data['last_contact_month'] = train_data['last contact date'].dt.month
    train_data['last_contact_year'] = train_data['last contact date'].dt.year
    
    test_data['last_contact_day'] = test_data['last contact date'].dt.day
    test_data['last_contact_month'] = test_data['last contact date'].dt.month
    test_data['last_contact_year'] = test_data['last contact date'].dt.year
    
    # Drop original 'last contact date' column
    train_data.drop(columns=['last contact date'], inplace=True)
    test_data.drop(columns=['last contact date'], inplace=True)





# Binary feature encoding
binary_columns = ['default', 'housing', 'loan']
for col in binary_columns:
    train_data[col] = train_data[col].map({'yes': 1, 'no': 0, 'unknown': -1})
    test_data[col] = test_data[col].map({'yes': 1, 'no': 0, 'unknown': -1})

# Ordinal encoding for education
education_map = {'unknown': 0, 'primary': 1, 'secondary': 2, 'tertiary': 3}
train_data['education'] = train_data['education'].map(education_map).astype(int)
test_data['education'] = test_data['education'].map(education_map).astype(int)
# train_data.drop(columns=['contact'], inplace=True)
# test_data.drop(columns=['contact'], inplace=True)
# One-Hot encoding for categorical variables
train_data = pd.get_dummies(train_data, columns=['job', 'marital', 'poutcome','contact'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['job', 'marital', 'poutcome','contact'], drop_first=True)



In [15]:
# categorical_columns = ['job', 'marital', 'default','education', 'housing', 'loan', 'contact', 'poutcome']
# numerical_columns = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
# train_data[categorical_columns] = train_data[categorical_columns].fillna('unknown').astype(str)
# test_data[categorical_columns] = test_data[categorical_columns].fillna('unknown').astype(str)

# train_data[numerical_columns] = train_data[numerical_columns].fillna(train_data[numerical_columns].mean())
# test_data[numerical_columns] = test_data[numerical_columns].fillna(test_data[numerical_columns].mean())

# education_map = {'unknown': 0, 'primary': 1, 'secondary': 2, 'tertiary': 3}
# train_data['education'] = train_data['education'].map(education_map)
# test_data['education'] = test_data['education'].map(education_map)

# # Define transformers for categorical and numerical columns
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
#     ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
# ])

# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# # Combine transformers into a ColumnTransformer for preprocessing
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numerical_columns + ['education']),
#         ('cat', categorical_transformer, categorical_columns)
#     ]
# )


In [16]:
# # Combine 'housing' and 'loan' into a single 'has_loan' column

# train_data['has_loan'] = train_data[['housing', 'loan']].apply(lambda x: 'yes' if 1 in x.values else ('no' if 0 in x.values else 'unknown'), axis=1)
# test_data['has_loan'] = test_data[['housing', 'loan']].apply(lambda x: 'yes' if 1 in x.values else ('no' if 0 in x.values else 'unknown'), axis=1)

# # Map 'has_loan' to 1 for 'yes', 0 for 'no', and -1 for 'unknown'
# train_data['has_loan'] = train_data['has_loan'].map({'yes': 1, 'no': 0, 'unknown': -1})
# test_data['has_loan'] = test_data['has_loan'].map({'yes': 1, 'no': 0, 'unknown': -1})


# train_data.drop(columns=['housing', 'loan'], inplace=True)
# test_data.drop(columns=['housing', 'loan'], inplace=True)


The Idea here was to combine the housing loan and loan but accuracy decreased

In [17]:

# # # Convert 'last contact date' to datetime format
# # train_data['last contact date'] = pd.to_datetime(train_data['last contact date'], errors='coerce')
# # test_data['last contact date'] = pd.to_datetime(test_data['last contact date'], errors='coerce')

# # # Calculate the number of days since last contact
# # current_date = datetime.today()
# # train_data['days_since_last_contact'] = (current_date - train_data['last contact date']).dt.days
# # test_data['days_since_last_contact'] = (current_date - test_data['last contact date']).dt.days

# # 
# # numerical_columns.append('days_since_last_contact')

#Align train and test sets
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)

# if 'last contact date' in train_data.columns:
#     train_data.drop(columns=['last contact date'], inplace=True)
#     test_data.drop(columns=['last contact date'], inplace=True)
    


In [18]:
X = train_data.drop(columns='target')
y = train_data['target'].map({'yes': 1, 'no': 0})  # Convert target to binary


# Save original column names for later use
# original_columns = X.columns


In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [20]:
# print(train_data.info())

In [21]:
# print(train_data.head())

In [22]:
# # Check the unique values in the 'has_loan' column
# print(X_train['has_loan'].value_counts())


In [23]:
# #correlation matrix 

# X_train_with_target = X_train.copy()
# X_train_with_target['target'] = y_train 

# corr_matrix_with_target = X_train_with_target.corr()

# plt.figure(figsize=(16, 10))
# sns.heatmap(corr_matrix_with_target, annot=True, cmap='coolwarm', linewidths=0.5)
# plt.title("Correlation Matrix Including Target Variable")
# plt.show()


In [24]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from xgboost import XGBClassifier
# from sklearn.metrics import f1_score

In [25]:
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Initialize SMOTE for oversampling
smote = SMOTE(random_state=42, k_neighbors=6)

# Fit SMOTE on the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Initialize LightGBM model
lgb_model = lgb.LGBMClassifier(random_state=42)

# Hyperparameters for tuning
param_grid_lgb = {
    'n_estimators': [102],
    'max_depth': [4],
    'num_leaves': [63],  # Add num_leaves to grid search
    'learning_rate': [0.1],
    'subsample': [0.8],
    'colsample_bytree': [ 1]
}
# {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'num_leaves': 63, 'subsample': 0.8}

# Perform Grid Search with Cross-Validation
grid_search_lgb = GridSearchCV(estimator=lgb_model, param_grid=param_grid_lgb,
                               scoring='f1_macro', cv=3, n_jobs=-1)
grid_search_lgb.fit(X_train_resampled, y_train_resampled)

# Print best parameters and best score
print("Best Parameters for LightGBM:", grid_search_lgb.best_params_)
print("Best Cross-Validated F1 Score for LightGBM:", grid_search_lgb.best_score_)

# Predict on validation set
y_val_pred = grid_search_lgb.predict(X_val_scaled)

# Evaluate model performance
val_f1_score = f1_score(y_val, y_val_pred, average='macro')
print("F1 Score on Validation Set:", val_f1_score)

# Make predictions on test data
# test_predictions = grid_search_lgb.predict(test_data)

# Create submission file
# submission = pd.DataFrame({
#     'id': test_data.index,
#     'target': ['yes' if pred == 1 else 'no' for pred in test_predictions]
# })

# # Save the submission
# submission.to_csv('submission.csv', index=False)
# print("Submission file created: submission.csv")


[LightGBM] [Info] Number of positive: 17826, number of negative: 17826
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010656 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4890
[LightGBM] [Info] Number of data points in the train set: 35652, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

In [26]:
best_params = grid_search_lgb.best_params_

# Print the best parameters
print("Best Parameters :", best_params)


Best Parameters : {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 102, 'num_leaves': 63, 'subsample': 0.8}


In [27]:
test_data.columns

Index(['age', 'education', 'default', 'balance', 'housing', 'loan', 'duration',
       'campaign', 'pdays', 'previous', 'target', 'last_contact_day',
       'last_contact_month', 'last_contact_year', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student', 'job_technician',
       'job_unemployed', 'job_unknown', 'marital_married', 'marital_single',
       'poutcome_other', 'poutcome_success', 'poutcome_unknown',
       'contact_telephone', 'contact_unknown'],
      dtype='object')

In [28]:
# Ensure both training and test data have the same columns after one-hot encoding
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)

# Ensure the 'target' column is not present in the test set (since it's for prediction)
test_data_features = test_data.drop(columns='target')

# Scale both training and test data (features only)
X_train_scaled = scaler.fit_transform(X_train)  # Fit and transform on training data
X_test_scaled = scaler.transform(test_data_features)  # Only transform on test data

# Make predictions on the test data
test_predictions = grid_search_lgb.predict(X_test_scaled)

# Create submission file
submission = pd.DataFrame({
    'id': test_data.index,  # Use index for test data IDs
    'target': ['yes' if pred == 1 else 'no' for pred in test_predictions]
})

# Save the submission
submission.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")


Submission file created: submission.csv


1. 

In [29]:
# # from sklearn.model_selection import GridSearchCV

# xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42)
# param_grid_xgb = {
#     'n_estimators': [100, 200],
#     'max_depth': [3, 6],
#     'learning_rate': [0.1, 0.01],
#     'subsample': [0.8, 1],
#     'colsample_bytree': [0.8, 1]
# }

# grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb,
#                                scoring='f1_macro', cv=3, n_jobs=-1)
# grid_search_xgb.fit(X_train_scaled, y_train)

# # Print best parameters
# print("Best Parameters for XGBoost:", grid_search_xgb.best_params_)
# print("Best Cross-Validated F1 Score for XGBoost:", grid_search_xgb.best_score_)



In [30]:
# y_val_pred_xgb = best_xgb_model.predict(X_val_scaled)

# # F1 Score for XGBClassifier
# f1_xgb = f1_score(y_val, y_val_pred_xgb, average='macro')

# # Confusion Matrix for XGBClassifier
# cm_xgb = confusion_matrix(y_val, y_val_pred_xgb)

# # Plot Confusion Matrix
# plt.figure(figsize=(6, 4))
# sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
# plt.title("Confusion Matrix for XGBClassifier")
# plt.ylabel('Actual')
# plt.xlabel('Predicted')
# plt.show()

# print(f"F1 Score for XGBClassifier: {f1_xgb}")


got 71.517

In [31]:
#Feature selection

In [32]:


# # Initialize the Logistic Regression model with L1 regularization (Lasso)
# lasso = LogisticRegression(penalty='l1', solver='liblinear', max_iter=2000, random_state=42, C=0.05)

# # Fit the model
# lasso.fit(X_train_scaled, y_train)

# # Select features with non-zero coefficients
# model = SelectFromModel(lasso, prefit=True)
# selected_features = X.columns[model.get_support()]  # Use the original column names

# print("Selected features:", selected_features)

In [33]:
# from sklearn.feature_selection import RFE
# # Feature Selection using Recursive Feature Elimination (RFE) with Logistic Regression
# log_reg = LogisticRegression(max_iter=2000, solver='lbfgs', random_state=42)
# rfe = RFE(estimator=log_reg, n_features_to_select=10) 
# rfe.fit(X_train_scaled, y_train)

# # Get selected feature names
# selected_features = X.columns[rfe.support_]
# print("Selected features:", selected_features)

In [34]:

# X_train_selected1 = X_train[selected_features]
# X_val_selected1 = X_val[selected_features]

# # Re-scale selected features
# X_train_selected = scaler.fit_transform(X_train_selected1)
# X_val_selected = scaler.transform(X_val_selected1)



# SGD Classifier model 

In [35]:

# # Initialize the SGDClassifier model
# sgd_model = SGDClassifier(random_state=42)
# # param_grid_sgd = {
# #     'loss': ['hinge', 'log'],
# #     'alpha': [0.001, 0.01, 0.1],
# #     'max_iter': [1000, 2000],
# #     'penalty': ['elasticnet', 'l2'],
# #     'eta0': [0.01, 0.1],
# #     'class_weight': ['balanced', None]
# # }

# param_grid_sgd = {
#     'loss': ['hinge'],
#     'alpha': [0.01],
#     'max_iter': [1000],
#     'penalty': ['elasticnet'],
#     'eta0': [0.01],  
#     'class_weight': ['balanced'] 
# }

# # Initialize GridSearchCV
# grid_search_sgd = GridSearchCV(estimator=sgd_model,  param_grid=param_grid_sgd,
#                                scoring='f1_macro',cv=3,
#                                 n_jobs=-1)

# # Fit the model
# grid_search_sgd.fit(X_train_selected, y_train)

# # Get the best parameters and score
# best_params_sgd = grid_search_sgd.best_params_
# best_score_sgd = grid_search_sgd.best_score_
# best_sgd_model = grid_search_sgd.best_estimator_

# print("Best Parameters for SGDClassifier:", best_params_sgd)
# print("Best Cross-Validated F1 Score for SGDClassifier:", best_score_sgd)


In [36]:
# # Apply feature selection to the validation set
# X_val_selected = X_val[selected_features]

# # Scale the validation set (using the same scaler as for the training set)
# X_val_selected_scaled = scaler.transform(X_val_selected)

# # Make predictions on the validation set
# test_predictions_val = best_sgd_model.predict(X_val_selected_scaled)

# # Calculate F1 Score on validation set
# f1_sgd_val = f1_score(y_val, test_predictions_val, average='macro')

# # Confusion Matrix for SGDClassifier on Validation Set
# cm_sgd_val = confusion_matrix(y_val, test_predictions_val)

# # Plot Confusion Matrix for validation set
# plt.figure(figsize=(6, 4))
# sns.heatmap(cm_sgd_val, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
# plt.title("Confusion Matrix for SGDClassifier (Validation Set)")
# plt.ylabel('Actual')
# plt.xlabel('Predicted')
# plt.show()

# print(f"F1 Score for SGDClassifier on Validation Set: {f1_sgd_val}")


In [37]:
# X_test_selected = test_data[selected_features]
# X_test_selected_scaled = scaler.transform(X_test_selected)
# test_predictions = best_sgd_model.predict(X_test_selected_scaled)


In [38]:
#Insights : Got the highest f1 score by the SGD model : 0.75675

# Random forest Classifier

In [39]:
# X = train_data.drop(columns='target')
# y = train_data['target'].map({'yes': 1, 'no': 0})

# # Step 1: Remove 'target' column if present in test_data
# if 'target' in test_data.columns:
#     test_data = test_data.drop(columns=['target'])


In [40]:
# from sklearn.model_selection import RandomizedSearchCV , cross_val_score 

# rf_model = RandomForestClassifier(random_state=42)
# # param_grid = {
# #     'n_estimators': [50, 100, 200, 300], 
# #     'max_depth': [5, 10, 15, 20, None], 
# #     'min_samples_split': [2, 5, 10, 20],  
# #     'min_samples_leaf': [1, 2, 4, 6],  
# #     'max_features': ['sqrt', 'log2', None], 
# #     'oob_score': [True, False],  
# #     'criterion': ['gini', 'entropy'], 
# #     'class_weight': [None, 'balanced'] 
# # }

# param_grid = {'oob_score': [True], 'n_estimators': [300], 'min_samples_split': [2], 
#               'min_samples_leaf': [6], 'max_features': [None], 'max_depth': [None], 
#               'criterion': ['gini'], 'class_weight': ['balanced']}

# grid_search = RandomizedSearchCV(
#     estimator=rf_model,
#     param_distributions=param_grid,
#     n_iter=10, 
#     cv=3,      
#     n_jobs=-1,  
#     random_state=42,
#     scoring='f1_macro' 
# )
# grid_search.fit(X_train_selected, y_train)

# print("Best hyperparameters :", grid_search.best_params_)

# # Get the best model from RandomizedSearchCV
# best_rf_model = grid_search.best_estimator_


# best_rf_model.fit(X_train_selected, y_train)


# y_val_pred = best_rf_model.predict(X_val_selected)

# # Evaluate the model on the validation set
# print("Random Forest Model Performance on Validation Set:")
# print("Accuracy:", accuracy_score(y_val, y_val_pred))
# print("Classification Report:\n", classification_report(y_val, y_val_pred))



In [41]:
# X_test_scaled = scaler.transform(test_data[selected_features])  # Scale the test set with selected features
# test_predictions = best_rf_model.predict(X_test_scaled)


In [42]:
#INsights : The f1_score for the test data fluctuated from 65% to 70% to this model after doing different changes

# KNN classifier

In [43]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.decomposition import PCA
# from sklearn.feature_selection import SelectKBest, f_classif

In [44]:


# model_pipeline = Pipeline([
#     ('feature_selection', SelectKBest(('pca', PCA(n_components=10, random_state=42)),  # PCA for f_classif, k=10)), 
# #     dimensionality reduction to 10 components
#     ('knn', KNeighborsClassifier(n_neighbors=5)) 
# ])


# model_pipeline.fit(X_train, y_train)

# # Step 4: Evaluate the model on validation data
# y_val_pred = model_pipeline.predict(X_val)

# # Print evaluation metrics
# print("K-Nearest Neighbors Model Performance on Validation Data:")
# print("Accuracy:", accuracy_score(y_val, y_val_pred))
# print("Classification Report:\n", classification_report(y_val, y_val_pred))
# test_predictions = model_pipeline.predict(test_data)

In [45]:
#insights : got approximately 70% accuracy thorugh this model

In [46]:
# if 'target' in test_data.columns:
#     test_data = test_data.drop(columns=['target'])
# X_test_scaled = scaler.transform(test_data)
# test_predictions = grid_search_sgd.predict(X_test_scaled)


In [47]:

# submission = pd.DataFrame({
#     'id': test_data.index,  
#     'target': ['yes' if pred == 1 else 'no' for pred in test_predictions]
# })

# # Save the submission
# submission.to_csv('submission.csv', index=False)
# print("Submission file created: submission.csv")

In [48]:
# import matplotlib.pyplot as plt

# # Model names and corresponding F1 scores
# models = ['SGD', 'Random Forest', 'XGBoost', 'KNN']
# f1_scores = [0.756, 0.68, 0.71517, 0.69]

# # Plotting the F1 scores as a bar graph
# plt.figure(figsize=(6, 4))
# plt.bar(models, f1_scores, color=['skyblue', 'lightgreen', 'coral', 'lightpink'])
# plt.xlabel("Models")
# plt.ylabel("F1 Score")
# plt.ylim(0, 1)
# plt.title("Comparison of F1 Scores for Different Models")
# plt.show()


In [49]:
# import pandas as pd
# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from imblearn.over_sampling import SMOTE
# import lightgbm as lgb
# from sklearn.metrics import classification_report, f1_score, roc_auc_score
# import matplotlib.pyplot as plt

In [50]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from imblearn.over_sampling import SMOTE
# import lightgbm as lgb
# from sklearn.metrics import classification_report, f1_score
# import matplotlib.pyplot as plt

# # Load your training data
# # data = pd.read_csv('dataset.csv')

# # Preprocessing: Extract year, month, and day from 'last contact date' if it exists
# if 'last contact date' in data.columns:
#     data['last contact date'] = pd.to_datetime(data['last contact date'], errors='coerce')
#     data['contact_year'] = data['last contact date'].dt.year
#     data['contact_month'] = data['last contact date'].dt.month
#     data['contact_day'] = data['last contact date'].dt.day
#     data.drop(columns=['last contact date'], inplace=True)

# # Separate features and target
# X = data.drop('target', axis=1)  # Replace 'target' with your actual target column name
# y = data['target']              # Replace 'target' with your actual target column name

# # Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Define categorical and numerical columns
# categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
# numerical_cols = [col for col in X_train.columns if col not in categorical_cols]

# # Preprocessing for numerical and categorical features
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean'))
# ])

# # Combine transformers in a ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)
#     ]
# )

# # Apply SMOTE to balance the classes
# smote = SMOTE(random_state=42)

# # Preprocess the training data and apply SMOTE
# X_train_preprocessed = preprocessor.fit_transform(X_train)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train)

# # Preprocess the validation data
# X_val_preprocessed = preprocessor.transform(X_val)

# # Define the LightGBM model
# lgbm_model = lgb.LGBMClassifier(random_state=42)

# # Train the LightGBM model
# lgbm_model.fit(X_train_resampled, y_train_resampled)

# # Make predictions on the validation set
# y_val_pred = lgbm_model.predict(X_val_preprocessed)

# # Evaluate the model on validation data
# print("Classification Report:")
# print(classification_report(y_val, y_val_pred))
# print("Macro F1 Score:", f1_score(y_val, y_val_pred, average='macro'))

# # Feature Importance Visualization
# plt.figure(figsize=(10, 6))
# lgb.plot_importance(lgbm_model, max_num_features=10, importance_type='split', figsize=(10, 6))
# plt.title("Feature Importance")
# plt.show()

# # Load and preprocess the test data
# # test_data = pd.read_csv('test_data.csv')

# # Apply the same preprocessing steps to the test data
# if 'last contact date' in test_data.columns:
#     test_data['last contact date'] = pd.to_datetime(test_data['last contact date'], errors='coerce')
#     test_data['contact_year'] = test_data['last contact date'].dt.year
#     test_data['contact_month'] = test_data['last contact date'].dt.month
#     test_data['contact_day'] = test_data['last contact date'].dt.day
#     test_data.drop(columns=['last contact date'], inplace=True)

# # Separate features from the test data (do not drop 'target' in test data as it doesn't exist)
# X_test = test_data

# # Preprocess the test data
# X_test_preprocessed = preprocessor.transform(X_test)

# # Make predictions on the test set
# test_predictions = lgbm_model.predict(X_test_preprocessed)

# # Assuming 'id' exists in test_data, or you can generate it based on the index
# submission = pd.DataFrame({
#     'id': test_data.index,  # Adjust based on your test data's unique identifier (if available)
#     'target': ['yes' if pred == 1 else 'no' for pred in test_predictions]
# })

# # Save the submission file
# submission.to_csv('submission.csv', index=False)
# print("Submission file created: submission.csv")


In [51]:
# if 'last contact date' in test_data.columns:
#     test_data['last contact date'] = pd.to_datetime(test_data['last contact date'], errors='coerce')
#     test_data['contact_year'] = test_data['last contact date'].dt.year
#     test_data['contact_month'] = test_data['last contact date'].dt.month
#     test_data['contact_day'] = test_data['last contact date'].dt.day
#     test_data.drop(columns=['last contact date'], inplace=True)

# # Separate features from the test data
# X_test = test_data.drop('target', axis=1)  # Ensure 'target' is not in the test data

# # Preprocess the test data
# X_test_preprocessed = preprocessor.transform(X_test)

# # Make predictions on the test set
# test_predictions = lgbm_model.predict(X_test_preprocessed)

# # Create the submission DataFrame
# submission = pd.DataFrame({
#     'id': test_data.index,  # or use 'id' column if it exists in the test dataset
#     'target': ['yes' if pred == 1 else 'no' for pred in test_predictions]
# })

# # Save the submission file
# submission.to_csv('submission.csv', index=False)
# print("Submission file created: submission.csv")